In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import uuid

from datasets import Dataset

In [22]:
def get_all_authors_composition():
    url = "http://www.litra.ru/composition/"
    response = requests.get(url=url)
    soup = BeautifulSoup(response.text)
    all_li = soup.find_all("li")

    base_url = "http://www.litra.ru"
    autors_compositions = [
        base_url + item.find("a")["href"]
        for item in all_li
        if "composition/work" in item.find("a")["href"]
    ]
    return autors_compositions


autors_compositions = get_all_authors_composition()
# autors_compositions = autors_compositions[:2]
# autors_compositions

In [23]:
def get_all_author_subworks(url: str = None):
    # url = "http://www.litra.ru/composition/work/wrid/00040701184773068573/"
    response = requests.get(url=url)
    soup = BeautifulSoup(response.text)
    all_table = soup.find(
        "body",
    )
    all_table = all_table.find_all("table", recursive=False)[1]
    base_url = "http://www.litra.ru"
    # all_table = all_table.find('tbody')#find('tr').find_all('td', recursive=False)
    all_table = all_table.find("tr").find_all("td", recursive=False)[1].find_all("a")
    all_table = [
        base_url + item["href"] for item in all_table if "woid" in item["href"]
    ]
    all_table = list(set(all_table))

    return all_table


# all_table
author_subworks = get_all_author_subworks(url=autors_compositions[1])
author_subworks

['http://www.litra.ru/composition/work/woid/00278901211194492103/',
 'http://www.litra.ru/composition/work/woid/00082201184773070406/']

In [24]:
def get_work_link(url: str = None):
    # url = "http://www.litra.ru/composition/work/woid/00855211223738000477/"
    base_url = "http://www.litra.ru"
    response = requests.get(url=url)
    soup = BeautifulSoup(response.text)
    all_table = soup.find(
        "body",
    )
    all_table = all_table.find_all("table", recursive=False)[1]
    # all_table = all_table.find('tbody')#find('tr').find_all('td', recursive=False)
    all_table = all_table.find("tr").find_all("td", recursive=False)[1].find_all("a")
    all_table = [
        base_url + item["href"] for item in all_table if "coid" in item["href"]
    ]
    all_table = list(set(all_table))

    return all_table


work_links = get_work_link(url=author_subworks[1])
work_links

['http://www.litra.ru/composition/get/coid/00024701184864249180/woid/00082201184773070406/']

In [49]:
def get_work_data(url: str = None):
    try:
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text)
        all_table = soup.find(
            "body",
        )
        all_table = all_table.find_all("table", recursive=False)[1]
        all_table = all_table.find("tr").find_all("td", recursive=False)[1]
        text = all_table.find_all("p")[1].text.strip().replace("\xa0", "")
        title = all_table.find_all("h1")[0].text.strip().replace("\xa0", "")

        return {
            "text": text,
            "title": title,
            "url": url,
        }
    except:
        print(url)
        return {
            "text": 'text',
            "title": 'title',
            "url": url,
        }
        


work_data = get_work_data(url=work_links[0])
print(work_data)

{'text': 'В предисловии к сборнику своей прозы «Круги на воде» (1912) Аверченко определил значение этой книги, да и всего своего творчества, весьма остроумно: «Прямо спасение утопающих!». Писатель имел в виду угрозу захлебнуться в потоке мрачного пессимизма, наполнившего литературу того времени. Но в этом литературном девизе обозначена и важная для писателя художественная задача. Что бы не высмеивал Аверченко, с какой бы бескомпромиссностью не обличал зло, его произведения всегда пронизаны острым комизмом, вызывающим веселый смех. Аверченко обязательно излечивал безысходную тоску, и делал это с редкой легкостью и изобретательностью.\nА.Т. Аверченко был редактором журнала «Новый сатирикон» и самым плодовитым его автором. Его рассказы и юморески часто занимали чуть ли не половину номера. Из них составлялись сборники рассказов Аверченко: «Веселые устрицы», «Зайчики на стене», (1910), «Круги на воде» (1912), «Сорные травы» (1914) и другие.\nОбъектами сатиры Аверченко являлись повседневные 

In [48]:
def get_all_author_works(url: str = None):
    author_subworks = get_all_author_subworks(url=url)
    all_work_links = []
    for author_subwork in author_subworks:
        work_links = get_work_link(url=author_subwork)
        all_work_links.extend(work_links)

    all_work_links_filtered = []
    uuid_works = set()
    for work_link in all_work_links:
        uuid_work = [item.strip() for item in work_link.split("/") if len(item) > 0][-1]
        if not uuid_work in uuid_works:
            all_work_links_filtered.append(work_link)
            uuid_works.add(uuid_work)

    all_author_data = []
    for work_link in tqdm(all_work_links_filtered):
        work_data = get_work_data(url=work_link)
        all_author_data.append(work_data)

    return all_author_data


all_author_works = get_all_author_works(url=autors_compositions[0])

100%|██████████| 6/6 [00:00<00:00,  6.30it/s]


In [41]:
all_author_works = Dataset.from_list(all_author_works)

In [42]:
all_author_works

Dataset({
    features: ['text', 'title', 'url'],
    num_rows: 6
})

In [44]:
all_author_works[1]["text"]

'Недавно я прочитал книгу. Она называется "Приключения Тома Сойера", её автор Марк Твен. Эту повесть Твен напечатал в 1876 году. Когда она вышла в свет, ее успех превзошел все ожидания. Но и в наше время повесть остается быть актуальной. Уже более ста лет ее охотно читают. Надо отметить, что прообразом главного героя стал сам писатель. "Приключения Тома Сойера" - одна из самых веселых книг. В неи описываются многие смешные истории школьной жизни Тома. Например, когда подшутили над лысым учителем, незаметно сняв с него парик. Также в книге происходят и нешуточные испытания, такие как история с Индейцем Джо, как Том и Бекки потерялись в пещере.\nСамым главным открытием для меня стала настоящая дружба Тома и Гека. Интересно то, что двое мальчишек проявляют настоящие мужские чувства по отношению друг к другу. Том и Гек настоящие авантюристы. Они попадают в самые нелепые ситуации благодаря своему неудержимому любопытству.\nВ повести присутствует и романтика, это отношения Тома и Бекки, он у

In [45]:
all_author_works[1]["title"]

'Отзыв по книге "Приключения Тома Сойера"'

In [50]:
all_authors_composition = get_all_authors_composition()

all_author_works = []
for authors_composition in tqdm(all_authors_composition):
    all_author_work = get_all_author_works(url=authors_composition)
    all_author_works.extend(all_author_work)

100%|██████████| 257/257 [04:29<00:00,  1.05s/it]


In [55]:
all_author_works[:5]

[{'text': 'Есть женщины в русских селеньях... \nН. А. Некрасов\nПовести Федора Абрамова — “Деревянные кони”, “Пелагея” и “Алька” — были завершены почти одновременно — в 1969 и 1971 годах. Писатель придал им особое значение.\nВ этих повестях воплощена история русской деревни, многострадальной жизни крестьян и прежде всего русской женщины.\nТрилогия начинается повестью “Деревянные кони”. В ней рассказывается о жизни Милентьевны, русской крестьянки. Про жизнь ее мы узнаем из рассказов Евгении — невестки Милентьевны. И жизнь эта была далеко не легкой. Шестнадцати лет Милентьевну замуж выпихнули. От рассвета до заката — непосильная работа, заботы по дому. Двух сыновей на войне убили. Но выстояла Милентьевна, выдержала все невзгоды. И даже теперь, несмотря на свою старость, не могла сидеть без работы. Каждое утро уходила в лес за грибами. Возвращалась еле живая, но не хотела покоряться усталости, немощи и возрасту. (А шел Милентьевне уже седьмой десяток.) Однажды пришла она совсем больная и 

In [52]:
all_author_works_dataset = Dataset.from_list(all_author_works)

In [53]:
all_author_works_dataset

Dataset({
    features: ['text', 'title', 'url'],
    num_rows: 650
})

In [54]:
all_author_works_dataset.push_to_hub("dim/litra_ru_essays")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:57<00:00, 57.38s/it]
